<a href="https://colab.research.google.com/github/bomb1000/new_keywords_analysis/blob/main/%E6%96%B0%E8%81%9E%E8%AA%9E%E6%96%99%E9%97%9C%E9%8D%B5%E5%AD%97%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>